In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

df = pd.DataFrame(columns=['Название', 'Марка', 'Модель', 'Пробег, км', 'Мощность двигателя, л',
                         'Мощность двигателя, л.с', 'Год выпуска', 'Цена, руб', 'Цвет кузова', 
                         'Коробка передач','Привод', 'Тип топлива','Город продажи'])

def safe_text(element):
    if element:
        return element.text.strip()
    return "Not available"

def safe_get_from_list(items, index, default="Not available"):
    try:
        if items and len(items) > index:
            return items[index].text.strip()
        return default
    except:
        return default

for page in range(1, 99):
    try:
        url = f"https://auto.ru/moskva/cars/vendor-chinese/all/?catalog_filter=mark%3DCHERY&catalog_filter=mark%3DGEELY&catalog_filter=mark%3DHAVAL&page={page}"
        cookie = 'layout-config={\"screen_height\":1440,\"screen_width\":2560,\"win_width\":2545,\"win_height\":861}; from=direct; from_lifetime=1684001441347; gdpr=0; sso_status=sso.passport.yandex.ru:blocked; count-visits=11; _ym_d=1684001438; _ym_uid=1681200562636975776; safeframe-test-cookie_1684001438875_84_1=test; cycada=YSLSvbFiQygcruAbWj+TKQcmfcvC6eJUR+1FBVDSUlA=; _yasc=E2S73mQ7CwdlfpzTnefbdItMkxozenNwFkuUmRn3NVPhN2lgt0mWSeQqi2sT1g==; spravka=dD0xNjgzOTk2MzgyO2k9ODcuMjQwLjQ5LjI1MTtEPTY3M0QwODBDMTYwMkU3MEU3QzEwMzNGQkNENDA4NjFCNDAwMjdFNzlBQkQ0Mzc5NEUxMTZCRkMxMDY1RDFGNDQyRTJENjVFMzg2ODAwMjt1PTE2ODM5OTYzODIyODU5MDgxNzU7aD1iZjliMzAxNWVkMTMwYTM0Y2JiNTRlZjg0YTIzOTk2Zg==; popups-popup-pdd-spring-shown-count=1; listing_view=%7B%22output_type%22%3Anull%2C%22version%22%3A1%7D; listing_view_session={}; autoru-visits-count=1; gids=; gradius=200; L=cSR0Q3RjQX9DYEVXV0FbR0BuQV9idnhaJUE3AkotOFQ4IgIuEwZHARtRGCYA.1680275506.15298.330958.a60cf2711ba152838a1f7f495a70fb03; mda2_beacon=1683992077143; yandex_login=asbryuzgin@edu.hse.ru; ys=udn.cDphc2JyeXV6Z2luQGVkdS5oc2UucnU%3D#c_chck.3556194812; _ym_isad=2; autoru_sso_blocked=1; _csrf_token=f918fad396081a0fcbaa7e0cb59b54601eb972fd602e29de; my=YwA%3D; yandexuid=7636177771605955887; yuidlt=1; suid=8cde0a8b0d828f76b9479e2764000e9e.3851a07104d8ccc12fdc6e158c6896db'
        headers = {
            'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 16_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 YaBrowser/23.3.4.311.11 Mobile/15E148 Safari/604.1',
            'Accept': '*/*', 
            'Connection': 'keep-alive',
            'Referer': 'https://www.google.com/',
            'Cache-Control': 'no-cache',
            'Upgrade-Insecure-Requests': '1',
            'If-None-Match': 'W/"2a-nG1r6lLq3nAKWlU+/B6Uhx9pU1c"',
            'Cookie': cookie 
        }
        
        proxies = {'https': '135.181.101.187:8080'}
        response = requests.get(url, headers=headers)
        
        if response.status_code != 200:
            print(f"Error accessing page {page}: Status code {response.status_code}")
            time.sleep(10)  
            continue
            
        soup = BeautifulSoup(response.content, "html.parser")
        
        no_results = soup.find('div', {'class': 'NoSearchResults'})
        if no_results:
            print(f"No more results found at page {page}. Stopping.")
            break

        cars = soup.find_all('div', {'class': 'ListingItem__main'})
        print(f"Found {len(cars)} car listings on page {page}")
        
        if not cars:
            print(f"No car listings found on page {page}. Checking for CAPTCHA or site changes...")
            with open(f'page_{page}_debug.html', 'w', encoding='utf-8') as f:
                f.write(str(soup))
            if page > 1:  
                time.sleep(30)
                continue
        
        for car in cars:
            try:
                title_element = car.find('a', {'class': 'Link ListingItemTitle__link'})
                name = safe_text(title_element)
                
                if not name or name == "Not available":
                    print("Missing title for car, skipping...")
                    continue
                    
                brand = name.split()[0] if name and name != "Not available" else "Not available"
                model = ' '.join(name.split()[1:]) if name and name != "Not available" and len(name.split()) > 1 else "Not available"
                
                km_element = car.find('div', {'class': 'ListingItem__kmAge'})
                km = safe_text(km_element).replace('км', '').strip() if km_element else "Not available"
            
                year_element = car.find('div', {'class': 'ListingItem__year'})
                year = safe_text(year_element)
                
                price_element = car.find('div', {'class': 'ListingItemPrice__content'})
                price = safe_text(price_element).replace('₽', '').strip() if price_element else "Not available"
                
                location_element = car.find('span', {'class': 'MetroListPlace__regionName MetroListPlace_nbsp'})
                location = safe_text(location_element)
                
                TechSummary = car.find('div', {'class': 'ListingItemTechSummaryDesktop'})
                tech_cells = []
                if TechSummary:
                    tech_cells = TechSummary.find_all('div', {'class': 'ListingItemTechSummaryDesktop__cell'})
                
                power = "Not available"
                horse_power = "Not available"
                if tech_cells and len(tech_cells) > 0:
                    try:
                        first_cell_text = tech_cells[0].text
                        if '\u2009' in first_cell_text:
                            parts = first_cell_text.split('\u2009')
                            power = parts[0].replace('л', '').strip() if len(parts) > 0 else "Not available"
                            horse_power = parts[2].replace('\xa0л.с.', '').strip() if len(parts) > 2 else "Not available"
                    except Exception as e:
                        print(f"Error extracting power: {e}")
                
                fuel = "Not available"
                if tech_cells and len(tech_cells) > 0:
                    try:
                        first_cell_text = tech_cells[0].text
                        if '\u2009' in first_cell_text:
                            parts = first_cell_text.split('\u2009')
                            fuel = parts[4] if len(parts) > 4 else "Not available"
                    except Exception as e:
                        print(f"Error extracting fuel type: {e}")
                
                color = safe_get_from_list(tech_cells, 4)
                transmission = safe_get_from_list(tech_cells, 1)
                drivetrain = safe_get_from_list(tech_cells, 3)
                
                new_row = {
                    'Название': name, 
                    'Марка': brand, 
                    'Модель': model, 
                    'Пробег, км': km, 
                    'Мощность двигателя, л': power,
                    'Мощность двигателя, л.с': horse_power, 
                    'Год выпуска': year, 
                    'Цена, руб': price,
                    'Цвет кузова': color, 
                    'Коробка передач': transmission, 
                    'Привод': drivetrain, 
                    'Тип топлива': fuel, 
                    'Город продажи': location
                }
                
                df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
                
            except Exception as e:
                print(f"Error processing a car listing: {e}")
                continue
        
        if page % 5 == 0:
            df.to_csv(f'chinese_cars_checkpoint_page_{page}.csv', index=False, encoding='utf-8-sig')
            print(f"Checkpoint saved at page {page}, cars collected so far: {len(df)}")
            
        print(f"Completed page {page}")
        
        sleep_time = 3 + 4 * (page % 2) 
        time.sleep(sleep_time)
        
    except Exception as e:
        print(f"Error processing page {page}: {e}")
        time.sleep(15) 
        continue

print(f"Total cars scraped: {len(df)}")
df.to_csv('chinese_cars_final.csv', index=False, encoding='utf-8-sig')
print("Data saved to chinese_cars_final.csv")
df


Found 0 car listings on page 1
No car listings found on page 1. Checking for CAPTCHA or site changes...
Completed page 1
Found 38 car listings on page 2
Completed page 2
Found 37 car listings on page 3
Completed page 3
Found 38 car listings on page 4
Completed page 4
Found 38 car listings on page 5
Checkpoint saved at page 5, cars collected so far: 151
Completed page 5
Found 37 car listings on page 6
Completed page 6
Found 38 car listings on page 7
Completed page 7
Found 38 car listings on page 8
Completed page 8
Found 38 car listings on page 9
Completed page 9
Found 38 car listings on page 10
Checkpoint saved at page 10, cars collected so far: 340
Completed page 10
Found 38 car listings on page 11
Completed page 11
Found 37 car listings on page 12
Completed page 12
Found 37 car listings on page 13
Completed page 13
Found 38 car listings on page 14
Completed page 14
Found 37 car listings on page 15
Checkpoint saved at page 15, cars collected so far: 527
Completed page 15
Found 38 car l

,Название,Марка,Модель,"Пробег, км","Мощность двигателя, л","Мощность двигателя, л.с",Год выпуска,"Цена, руб",Цвет кузова,Коробка передач,Привод,Тип топлива,Город продажи
0,Chery Tiggo 8 Pro Max I Рестайлинг,Chery,Tiggo 8 Pro Max I Рестайлинг,Новый,2.0,197,2024,3 870 000,Dreamline 4WD,робот,полный,Бензин,Москва
1,Chery Tiggo 7 Pro I,Chery,Tiggo 7 Pro I,143 900,1.5,147,2021,1 380 000,чёрный,вариатор,передний,Бензин,Москва
2,Haval M6 II,Haval,M6 II,Новый,1.5,143,2024,от 1 526 000,Family,механика,передний,Бензин,Москва
3,Geely Preface I,Geely,Preface I,Новый,2.0,200,2024,от 2 607 990,Luxury,робот,передний,Бензин,Москва
4,Geely Monjaro (Xingyue L) I,Geely,Monjaro (Xingyue L) I,21 700,2.0,238,2023,3 000 000,белый,автомат,полный,Бензин,Москва
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3635,Geely Icon,Geely,Icon,1,1.5,181,2024,2 699 999,голубой,робот,передний,Бензин,Москва
3636,Geely Icon,Geely,Icon,10,1.5,181,2024,2 699 999,серый,робот,передний,Бензин,Москва
3637,Chery Tiggo 8 Pro I,Chery,Tiggo 8 Pro I,50,2.0,170,2023,3 050 000,серый,вариатор,передний,Бензин,Москва
3638,Geely Atlas Pro,Geely,Atlas Pro,58 000,1.5,177,2022,2 100 000,синий,робот,полный,Бензин,Одинцово
